<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Turku_Finland_MeteoSat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import requests
import pandas as pd
import time

# --- CONFIGURATION ---
# Turku, Finland
LAT = 60.4518
LON = 22.2666

# Full Year 2024
START_DATE = "2024-01-01"
END_DATE = "2025-01-01"

# API Endpoint (Historical Weather)
url = "https://archive-api.open-meteo.com/v1/archive"

# Parameters:
# shortwave_radiation = GHI (Global Horizontal Irradiance)
# direct_normal_irradiance = DNI
# diffuse_radiation = DHI
params = {
    "latitude": LAT,
    "longitude": LON,
    "start_date": START_DATE,
    "end_date": END_DATE,
    "hourly": "temperature_2m,wind_speed_10m,shortwave_radiation,direct_normal_irradiance,diffuse_radiation",
    "timezone": "UTC"
}

print(f"Fetching Satellite-Derived Data for Turku ({START_DATE} to {END_DATE})...")

try:
    r = requests.get(url, params=params)

    if r.status_code == 200:
        data = r.json()

        # Check if hourly data exists
        if 'hourly' in data:
            hourly = data['hourly']

            df = pd.DataFrame({
                'Time': hourly['time'],
                'Temperature_C': hourly['temperature_2m'],
                'Wind_Speed_ms': hourly['wind_speed_10m'],
                'GHI_Wm2': hourly['shortwave_radiation'],
                'DNI_Wm2': hourly['direct_normal_irradiance'],
                'DHI_Wm2': hourly['diffuse_radiation']
            })

            # Formatting
            df['Time'] = pd.to_datetime(df['Time'])
            df['Date'] = df['Time'].dt.strftime('%Y-%m-%d')
            df['Hour'] = df['Time'].dt.strftime('%H:%M')

            # Select and Reorder columns
            cols = ['Date', 'Hour', 'Temperature_C', 'Wind_Speed_ms', 'GHI_Wm2', 'DNI_Wm2', 'DHI_Wm2']
            df = df[cols]

            print("-" * 30)
            print(df.head())
            print("-" * 30)
            print(f"Total Rows: {len(df)}")

            filename = "Turku_Meteosat_2024_Full.csv"
            df.to_csv(filename, index=False)
            print(f"Success! Data saved to {filename}")
        else:
            print("API returned 200 OK but no 'hourly' data found.")
    else:
        print(f"Error {r.status_code}: {r.text}")

except Exception as e:
    print(f"Connection failed: {e}")

Fetching Satellite-Derived Data for Turku (2024-01-01 to 2025-01-01)...
------------------------------
         Date   Hour  Temperature_C  Wind_Speed_ms  GHI_Wm2  DNI_Wm2  DHI_Wm2
0  2024-01-01  00:00          -12.2            8.0      0.0      0.0      0.0
1  2024-01-01  01:00          -13.7            7.0      0.0      0.0      0.0
2  2024-01-01  02:00          -12.7            8.3      0.0      0.0      0.0
3  2024-01-01  03:00          -12.1            9.8      0.0      0.0      0.0
4  2024-01-01  04:00          -11.5           11.5      0.0      0.0      0.0
------------------------------
Total Rows: 8808
Success! Data saved to Turku_Meteosat_2024_Full.csv
